# 간편식 소비량 년도별 데이터 수집 및 전처리

## 모듈 설치

In [ ]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.25.2', '1.5.3')

## 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 준비 및 전처리

In [13]:
years_consume = pd.read_csv('/content/drive/MyDrive/멋사/Team Space/data/raw_data/간편식_지출액.csv', encoding='cp949') # 데이터 불러오기

years_consume = years_consume.drop('특성별(1)', axis=1) # 특성별(1) 열 삭제
years_consume = years_consume.rename(columns={'특성별(2)': 'year'}) # 열 이름 변경
years_consume.columns = years_consume.columns.map(lambda x: x.split('.')[0]) # 연도 '.' 뒤에 삭제
drop_index_list = [8, 9] + list(range(23, 31)) # 필요없는 행 삭제(읍, 동, 학력, 직업)
years_consume = years_consume.drop(drop_index_list)
years_consume = years_consume.T.reset_index() # 행렬 변환
years_consume.head(10)

string_to_remove = '-' # 결측값 '-'으로 표시되어 있음
years_consume = years_consume.applymap(lambda x: x.replace(string_to_remove, '0')) # '-'값 찾아서 '0'으로 변환 -> 뒤에 계산해야 함.
years_consume.columns = years_consume.iloc[0]
years_consume = years_consume.drop(years_consume.index[0])
years_consume = years_consume.rename(columns={'특성별(2)': '항목'})

for col in years_consume.columns[2:]: # 2번쨰 열('소계')부터
    years_consume[col] = pd.to_numeric(years_consume[col]) # float 형식으로 변환

# 열 이름 통합
# 가구원수별 통합
years_consume['4인 이상'] = (years_consume['4인'] + years_consume['4인 이상'] + years_consume['5인 이상']) / 3 # 4인, 4인 이상, 5인 이상의 열 값 더하고 나누기 3 -> 다시 4인 이상으로 지정하여 저장
years_consume.drop(columns=['4인'], inplace=True) # 기존 열 삭제
years_consume.drop(columns=['5인 이상'], inplace=True) # 기존 열 삭제
# years_consume.head(10)

years_consume['200~400만원 미만'] = (years_consume['200~300만원 미만'] + years_consume['200~400만원 미만'] + years_consume['300~400만원 미만']) / 3 # 4인, 4인 이상, 5인 이상의 열 값 더하고 나누기 3 -> 다시 4인 이상으로 지정하여 저장
years_consume.drop(columns=['200~300만원 미만'], inplace=True) # 기존 열 삭제
years_consume.drop(columns=['300~400만원 미만'], inplace=True) # 기존 열 삭제

years_consume['400~600만원 미만'] = (years_consume['400~500만원 미만'] + years_consume['400~600만원 미만'] + years_consume['500~600만원 미만']) / 3 # 4인, 4인 이상, 5인 이상의 열 값 더하고 나누기 3 -> 다시 4인 이상으로 지정하여 저장
years_consume.drop(columns=['400~500만원 미만'], inplace=True) # 기존 열 삭제
years_consume.drop(columns=['500~600만원 미만'], inplace=True) # 기존 열 삭제

years_consume = years_consume.apply(lambda x: x.round(2) if x.dtype=='float' else x) # 둘째 자리에서 반올림

years_consume

,year,항목,소계,수도권,충청권,호남권,대경권,동남권,강원권,1인,...,여성,20대,30대,40대,50대,60대 이상,200만원 미만,200~400만원 미만,400~600만원 미만,600만원 이상
1,2018,1만원 미만 (%),2.7,2.4,1.6,7.6,2.2,1.2,2.3,3.3,...,4.4,1.6,1.7,1.9,2.6,4.7,7.1,1.80,0.97,0.9
2,2018,1만원~2만원 미만 (%),8.1,5.8,11.5,19.6,6.6,5.4,10.2,10.9,...,13.0,4.2,6.9,4.5,7.5,14.7,16.5,6.07,2.93,4.8
3,2018,2만원~4만원 미만 (%),23.8,25.6,28.1,27.6,22.8,15.7,12.0,23.9,...,25.4,13.0,17.6,20.1,24.8,34.8,30.3,14.97,14.30,28.1
4,2018,4만원~6만원 미만 (%),19.7,16.6,31.3,20.4,19.7,21.8,15.3,19.9,...,21.9,24.9,20.2,19.9,19.5,17.7,16.6,13.60,13.67,18.7
5,2018,6만원~8만원 미만 (%),8.2,9.9,10.1,4.7,1.8,9.1,1.9,8.1,...,6.8,7.4,9.7,8.3,9.2,5.9,4.2,5.77,5.23,12.8
6,2018,8만원~10만원 미만 (%),5.4,6.9,5.4,1.8,4.3,4.4,2.0,5.2,...,4.8,8.5,4.9,6.4,4.8,4.3,3.8,3.37,3.80,7.8
7,2018,10만원 이상 (%),32.1,32.7,12.0,18.2,42.6,42.5,56.4,28.6,...,23.7,40.4,38.9,38.9,31.5,17.8,21.4,21.10,25.77,26.9
8,2018,평균 (원),78875.0,80415.0,52938.0,50743.0,107401.0,84930.0,123370.0,71517.0,...,63884.0,87412.0,93353.0,95116.0,75739.0,51592.0,53802.0,50880.33,62570.33,73904.0
9,2019,1만원 미만 (%),0.6,0.7,0.4,0.8,0.2,0.3,0.0,1.2,...,1.1,0.0,0.4,0.5,1.0,1.6,1.7,0.30,0.20,0.0
10,2019,1만원~2만원 미만 (%),3.4,4.0,0.7,5.9,1.9,2.2,4.0,3.8,...,4.2,3.8,2.5,2.0,4.3,10.6,5.2,2.23,1.87,2.5


In [ ]:
years_consume

,특성별(1),특성별(2),2018,2018.1,2018.2,2018.3,2018.4,2018.5,2018.6,2018.7,...,2021.7,2022,2022.1,2022.2,2022.3,2022.4,2022.5,2022.6,2022.7,2022.8
0,특성별(1),특성별(2),1만원 미만 (%),1만원~2만원 미만 (%),2만원~4만원 미만 (%),4만원~6만원 미만 (%),6만원~8만원 미만 (%),8만원~10만원 미만 (%),10만원 이상 (%),평균 (원),...,평균 (원),1만원 미만 (%),1만원~2만원 미만 (%),2만원~4만원 미만 (%),4만원~6만원 미만 (%),6만원~8만원 미만 (%),8만원~10만원 미만 (%),10만원 이상 (%),무응답 (%),평균 (원)
1,전체,소계,2.7,8.1,23.8,19.7,8.2,5.4,32.1,78875,...,159629,0.5,4.4,18.3,20.8,11.1,6.3,38.1,0.4,91858
2,거주지역별,수도권,2.4,5.8,25.6,16.6,9.9,6.9,32.7,80415,...,217515,0.1,2.8,14.5,18.8,10.6,7.2,45.1,0.9,105294
3,거주지역별,충청권,1.6,11.5,28.1,31.3,10.1,5.4,12.0,52938,...,114041,0.3,3.7,11.6,25.6,14.6,6.4,37.8,0.0,97311
4,거주지역별,호남권,7.6,19.6,27.6,20.4,4.7,1.8,18.2,50743,...,85215,1.3,4.3,26.6,28.3,8.3,3.0,28.1,0.0,75993
5,거주지역별,대경권,2.2,6.6,22.8,19.7,1.8,4.3,42.6,107401,...,119787,0.5,5.0,16.1,15.0,12.6,8.5,42.3,0.0,83179
6,거주지역별,동남권,1.2,5.4,15.7,21.8,9.1,4.4,42.5,84930,...,96245,1.2,8.4,31.0,23.5,12.4,4.4,19.2,0.0,60595
7,거주지역별,강원권,2.3,10.2,12.0,15.3,1.9,2.0,56.4,123370,...,101745,2.0,10.8,21.5,14.2,5.5,5.8,40.2,0.0,91036
8,행정구역별,동,2.4,7.7,22.7,19.6,8.2,5.6,33.7,80667,...,164015,0.5,4.1,17.9,19.9,10.8,6.4,40.0,0.5,94900
9,행정구역별,읍면,3.9,10.0,28.7,20.1,7.8,4.5,25.0,70973,...,135809,0.6,6.0,20.9,25.8,12.8,5.9,28.0,0.0,75236


## 완료 데이터 csv 저장

In [ ]:
years_consume.to_csv('간편식_소비량_년도별.csv', encoding='cp949', index=False) # csv 파일 저장. index 삭제